In [ ]:
# This notebook contains some drafts of a high level API where arrays can be passed instead of widgets.
# It doesn't work. Leaving this for later, may also decide to abandon it because it will be inefficient to use.

In [ ]:
import numpy as np
from unray import *
from ipydatawidgets import *
import pythreejs as three

default_density_lut = np.linspace(0.0, 1.0, 256, dtype='float32')

default_emission_lut = np.outer(
    np.linspace(0.0, 1.0, 256, dtype='float32'),
    np.ones(3, dtype="float32")
    )

# def as_array_widget(values):
#     if values is None:
#         return None
#     if isinstance(values, NDArrayWidget):
#         return values
#     return NDArrayWidget(values)


def surf(field, lut=None, restrict=None):
    "TODO: Design and document."
    if lut is None:
        lut = ArrayColorLUT(values=[[0,0,0], [1,1,1]])
    color = ColorField(field=field, lut=lut)
    kwargs = dict(color=color, restrict=restrict, wireframe=False)
    return SurfacePlot(**kwargs)


def wireframe(field, lut=None, restrict=None):
    "TODO: Design and document."
    if lut is None:
        lut = ArrayColorLUT(values=[0,0,0, 1,1,1])
    color = ColorField(field=field, lut=lut)
    kwargs = dict(color=color, restrict=restrict, wireframe=True)
    return SurfacePlot(**kwargs)


def xray(field, restrict=None):
    "TODO: Design and document."
    kwargs = dict(density=field, restrict=restrict)
    return XrayPlot(**kwargs)


def minproj(field, lut=None, restrict=None):
    "TODO: Design and document."
    if lut is None:
        lut = ArrayColorLUT(values=[0,0,0, 1,1,1])
    color = ColorField(field=field, lut=lut)
    kwargs = dict(color=color, restrict=restrict)
    return MinProjPlot(**kwargs)


def maxproj(field, lut=None, restrict=None):
    "TODO: Design and document."
    if lut is None:
        lut = ArrayColorLUT(values=[0,0,0, 1,1,1])
    color = ColorField(field=field, lut=lut)
    kwargs = dict(color=color, restrict=restrict)
    return MaxProjPlot(**kwargs)


def sumproj(field, lut=None, restrict=None):
    "TODO: Design and document."
    if lut is None:
        lut = ColorLUTArrayColorLUTT(values=[0,0,0, 1,1,1])
    color = ColorField(field=field, lut=lut)
    kwargs = dict(color=color, restrict=restrict)
    return SumProjPlot(**kwargs)


def volume(field, restrict=None):
    "TODO: Design and document."
    lut0 = ScalarLut(values=[0,0,0, 1,1,1])
    density = ScalarField(field=field, lut=lut0)
    lut1 = ArrayColorLUT(values=[0,0,0, 1,1,1])
    color = ColorField(field=field, lut=lut)
    kwargs = dict(density=density, color=color, restrict=restrict)
    return VolumePlot(**kwargs)

In [ ]:
# Setup some data for testing

def single_tetrahedron():
    cells = np.zeros((1, 4), dtype="uint32")
    coordinates = np.zeros((4, 3), dtype="float32")
    cells[0, :] = [0, 1, 2, 3]
    coordinates[0, :] = [0, 0, 0]
    coordinates[1, :] = [1, 0, 0]
    coordinates[2, :] = [0, 1, 0]
    coordinates[3, :] = [0, 0, 1]
    return coordinates, cells

filename = "data/heart.npz"
#filename = "data/brain.npz"
#filename = "data/aneurysm.npz"

mesh_data = np.load(filename)
cells_array = mesh_data["cells"].astype(np.int32)
points_array = mesh_data["points"].astype(np.float32)

# Coordinates of all vertices in mesh
x = list(points_array.T)  # x[2] = z coordinate array for all vertices

# Model center 3d vector
center = list(map(lambda x: x.mean(), x))

# Model min/max coordinates
bbox = list(map(lambda x: (x.min(), x.max()), x))

# Coordinates with origo shifted to center of model
xm = list(map(lambda x, mp: x - mp, x, center))

# Distance from model center
xd = np.sqrt(sum(map(lambda x: x**2, xm)))
radius = xd.max()

# Distance from center, normalized to max 1.0
func_dist = xd / radius

# A constant for all vertices
func_const = np.ones(x[0].shape)

# x coordinate
func_x = x[0]

# A wave pattern from the center of the model
freq = 4
func_wave = 2.0 + np.sin((freq * 2 * np.pi / radius) * xd)

In [ ]:
# pythreejs setup
width = 960
height = 320
camera = three.PerspectiveCamera(position=[10, 10, 10], aspect=width/height)
key_light = three.DirectionalLight(position=[0, 10, 10])
ambient = three.AmbientLight(intensity=0.5)
scene = three.Scene(children=[key_light, ambient, camera], background='gray')
controls = three.OrbitControls(camera)
renderer = three.Renderer(scene, camera, [controls],
                          width=width, height=height)
renderer.background = "#dddddd"
display(renderer)

In [ ]:
# Remove all unray plots from scene
scene.children = tuple(filter(lambda c: c._model_module != "unray", scene.children))

In [ ]:
mesh = Mesh(cells=cells_array, points=points_array)
plot = surf(mesh)
scene.add(plot)